# AdjSim Tutorial

AdjSim is an agent-based modelling engine. It allows users to define simulation environments through which agents interact through ability casting and timestep iteration. It is tailored towards allowing agents to behave intelligently through the employment of Reinforcement Learning techniques such as Q-Learning. 

This tutorial will enumerate many of AdjSim's features through the construction of a simulation where a group of agents will play a game of tag. Lets dive in!

---

## The Simulation Space

AdjSim simulations take place inside the fittingly named `Simulation` object.

Lets begin by importing our libararies.

In [2]:
import adjsim
import numpy as np # AdjSim also heavily relies on numpy. Its usage is recommended.

We will make a trivial empty simulation to display its usage.

In [3]:
sim = adjsim.core.Simulation()

# Interactive simulation method.
sim.start() # Begin the simulation.
sim.step() # Take one step.
sim.step(10) # Take 10 steps.
sim.end() # end the simulation.

# Batch simulation method.
sim.simulate(100) # Start, Simulate for 100 timesteps, End. All in one go.

Simulating timestep 100/100 - population: 0

Its a little lonely in that simulation, lets add an agent.

In [4]:
sim.agents.add(adjsim.core.Agent())
sim.agents.add(adjsim.core.Agent())

sim.simulate(100)

Simulating timestep 100/100 - population: 2 

Things are happening, but its a little hard to see. Lets visualize our simulation space. Simulations and agents have a hierarchal class structure. `VisualSimulation` inherits from `Simulation`, and can be used to display a 2D simulation environment.

In [5]:
sim = adjsim.core.VisualSimulation()
sim.simulate(50)

Simulating timestep 50/50 - population: 0 

Similarly, an appropriate `VisualAgent` must be present in the simulation space if it is to be seen. The agent inheritance hierarchy is as follows:

1. `Agent`: The base agent class. Contains the attributes that are minimally needed in order for an agent to be simulated.
2. `SpatialAgent`: Adds the presence of the `pos` Attribute, allowing for a 2D position to be associated with an agent.
3. `VisualAgent`: Adds the attributes needed for visualization on top of those of `SpatialAgent`. This is the minimum class needed for an agent to appear inside a `VisualSimulation`.

In [8]:
sim.agents.add(adjsim.core.VisualAgent(pos=np.array([1,1])))
sim.simulate(50)

Simulating timestep 500/500 - population: 3 

Now we should see our agent! Next, lets make it do things.

---

## Agents

Agents are simulation artifacts that manipulate the simulation environment. In the adjsim model, within each timestep agents take turns acting out their environment manipulations. One iteration of an agent's environment manipulations is known as an agent's __step__.

There are two major aspects to an agent's __step__: 

1. A set of __actions__. An __action__ defines one distinct user-defined set of computations that the agent manipulates its environment with. It is simply a python function. The actions we will be defining in our simulation of the game of tag will be a _move_ and a _tag_ action.
2. A decision module. Decision modules are AdjSim objects that an agent uses to choose which actions to perform, and in what order.

We'll start by making an agent that can _move_, but that can't _tag_. The _move_ action will allow an agent to move in a random direction bounded by a square arena.

In [12]:
# Constants.
ARENA_BOUND = 100
MOVE_DIST = 20

# Actions must follos the following signiature. 
# The Simulation object will be passed in as the 'simulation' parameter.
# The agent calling the action will be passed as the 'source' parameter.
def move(simulation, source):
    movement = (np.random.rand(2) - 0.5) * MOVE_DIST
    source.pos = np.clip(source.pos + movement, -ARENA_BOUND, ARENA_BOUND)
    source.step_complete = True

The final line, `source.step_complete = True`, lets the decision module know that no further actions can be completed in the agent's __step__ after the current one. 

Now let's create an agent that uses this action. The agent we will be creating will have a `RandomRepeatedCastDecision`. This decision module will randomly select actions and invoke them until the `source.step_complete` attribute is true. This should result in one cast of the above-defined _move_ action.

In [13]:
class Mover(adjsim.core.VisualAgent):
    def __init__(self, x, y):
        super().__init__(pos=np.array([x, y]))
        
        # Set the decision module.
        self.decision = adjsim.decision.RandomSingleCastDecision()

        # Populate the agent's action list.
        self.actions["move"] = move

Let's take it out for a spin.

In [14]:
sim = adjsim.core.VisualSimulation()
sim.agents.add(Mover(0, 0))

sim.simulate(50)

Simulating timestep 50/50 - population: 1 

We observe an agent moving in random directions in each timestep. Nice! Lets give it some friends.

In [15]:
class MoverSimulation(adjsim.core.VisualSimulation):
    def __init__(self):
        super().__init__()

        for i in range(5):
            for j in range(5):
                self.agents.add(Mover(20*i, 20*j))
                
sim = MoverSimulation()
sim.simulate(50)

Simulating timestep 50/50 - population: 25 

We should now see a group of 25 agents moving in a similar fashion to our first one. The basics are now covered. Lets put together our simulation of a game of tag.

---

## Putting It All Together

The following will describe our simulation of a game of tag.

In [16]:
# Reiterate imports.
import adjsim
import numpy as np
import sys

# Constants.
ARENA_BOUND = 100
TAG_DIST_SQUARE = 100
MOVE_DIST = 20

def move(simulation, source):
    movement = (np.random.rand(2) - 0.5) * MOVE_DIST
    source.pos = np.clip(source.pos + movement, -ARENA_BOUND, ARENA_BOUND)
    source.step_complete = True

def tag(simulation, source):  
    if not source.is_it:
        return

    # Find nearest neighbour.
    closest_distance = sys.float_info.max
    nearest_neighbour = None
    for agent in simulation.agents:
        if agent.id == source.id:
            continue

        distance = adjsim.utility.distance_square(agent, source)
        if distance < closest_distance:
            nearest_neighbour = agent
            closest_distance = distance

    if closest_distance > TAG_DIST_SQUARE:
        return

    assert nearest_neighbour

    # Perform Tag.
    nearest_neighbour.is_it = True
    nearest_neighbour.color = adjsim.color.RED_DARK # This will change the agent's visual color.
    nearest_neighbour.order = 1 # Order describes what order the agents will take their steps in the simulation loop.
    source.is_it = False
    source.order = 0 # Order describes what order the agents will take their steps in the simulation loop.
    source.color = adjsim.color.BLUE_DARK # This will change the agent's visual color.

class Tagger(adjsim.core.VisualAgent):

    def __init__(self, x, y, is_it):
        super().__init__()

        self.is_it = is_it
        self.color = adjsim.color.RED_DARK if is_it else adjsim.color.BLUE_DARK
        self.pos = np.array([x, y])
        self.decision = adjsim.decision.RandomSingleCastDecision()

        self.actions["move"] = move
        self.actions["tag"] = tag

        if is_it:
            self.order = 1 # Order describes what order the agents will take their steps in the simulation loop.


class TaggerSimulation(adjsim.core.VisualSimulation):

    def __init__(self):
        super().__init__()

        for i in range(5):
            for j in range(5):
                self.agents.add(Tagger(20*i, 20*j, False))

        self.agents.add(Tagger(10, 10, True))
        
sim = TaggerSimulation()
sim.simulate(50)

Simulating timestep 50/50 - population: 26 

And that's it! We should observe a game of tag being played where the 'it' agent is red. It will tag the agents around it when it has the chance. 

---

## Filling the Noggin

These agents act randomly. Let's fix that.